In [ ]:
!pip install tensorflow

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Bidirectional, LSTM, Dense
)
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [2]:
# --- 1. 하이퍼파라미터 및 설정 (4번과 거의 동일) ---
VOCAB_SIZE = 10000
MAX_LEN = 250
EMBED_DIM = 100
# (CNN 대신 LSTM 하이퍼파라미터)
LSTM_UNITS = 64 # LSTM의 은닉 상태(hidden state) 크기
NUM_CLASSES = 1
BATCH_SIZE = 64
EPOCHS = 10 # (EarlyStopping을 사용하므로 넉넉하게)


In [3]:

# --- 2. 데이터 준비 (4번과 100% 동일) ---

print(f"Loading IMDB data (Top {VOCAB_SIZE} words)...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=VOCAB_SIZE)

print("Padding sequences...")
x_train_pad = pad_sequences(x_train, maxlen=MAX_LEN, padding='post')
x_test_pad = pad_sequences(x_test, maxlen=MAX_LEN, padding='post')

print(f"Padded train data shape: {x_train_pad.shape}")


Loading IMDB data (Top 10000 words)...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Padding sequences...
Padded train data shape: (25000, 250)


In [4]:

# --- 3. 양방향 LSTM 모델 정의 ---
# (4번 과제의 build_text_cnn_model 함수와 비교해 보세요)

def build_bilstm_model():
  # 1. 입력 레이어 (동일)
  inputs = Input(shape=(MAX_LEN,), dtype='int32')

  # 2. 임베딩 레이어 (동일)
  # [BATCH_SIZE, MAX_LEN] -> [BATCH_SIZE, MAX_LEN, EMBED_DIM]
  embedding_layer = Embedding(
    input_dim=VOCAB_SIZE,
    output_dim=EMBED_DIM,
    input_length=MAX_LEN
  )
  embedded = embedding_layer(inputs)

  # 3. (핵심) 양방향 LSTM 레이어
  #
  # Keras의 Bidirectional 래퍼(wrapper)가 LSTM을 감싸
  # 정방향/역방향으로 2번 처리한 뒤, 그 결과를 '합쳐' 줍니다.
  # (CNN의 Conv1D + GlobalMaxPooling1D 부분을 대체함)
  bilstm_layer = Bidirectional(
    LSTM(units=LSTM_UNITS, dropout=0.2, recurrent_dropout=0.2)
  )(embedded)
  # 출력: [BATCH_SIZE, LSTM_UNITS * 2] (정방향 64, 역방향 64)
  # (만약 return_sequences=True를 썼다면 시퀀스 전체가 나옴)

  # 4. 최종 출력 레이어
  # (이진 분류이므로 sigmoid와 1개 유닛 사용)
  outputs = Dense(NUM_CLASSES, activation='sigmoid')(bilstm_layer)

  # 5. 모델 생성
  model = Model(inputs=inputs, outputs=outputs)
  return model

In [5]:

model = build_bilstm_model()

# --- 4. 모델 컴파일 (4번과 동일) ---
model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 250)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 250, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,084,609 (4.14 MB)

 Trainable params: 1,084,609 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:

# --- 5. 모델 학습 (EarlyStopping 적용) ---
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=1, # 1 Epoch 동안 val_loss 개선 없으면 중지
    restore_best_weights=True # 가장 좋았던 가중치로 복원
)

print("\n--- Bi-LSTM 모델 학습 시작 ---")
history = model.fit(
  x_train_pad,
  y_train,
  batch_size=BATCH_SIZE,
  epochs=EPOCHS,
  validation_data=(x_test_pad, y_test),
  callbacks=[early_stopping] # 조기 종료 적용
)

print("--- 모델 학습 완료 ---")


--- Bi-LSTM 모델 학습 시작 ---
Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 183s 457ms/step - accuracy: 0.6384 - loss: 0.6257 - val_accuracy: 0.8202 - val_loss: 0.4292
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 203s 459ms/step - accuracy: 0.8242 - loss: 0.4184 - val_accuracy: 0.7909 - val_loss: 0.5004
--- 모델 학습 완료 ---


In [7]:

# --- 6. (보너스) 새로운 리뷰 예측 ---
# (4번에서 사용한 함수와 100% 동일하게 작동합니다)

# Keras IMDB 데이터셋의 단어-인덱스 사전 로드
word_index = imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<pad>"] = 0
word_index["<start>"] = 1
word_index["<unk>"] = 2

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [8]:

def predict_sentiment_keras(model, text):
  tokens = text.lower().split()
  indices = [word_index.get(word, 2) for word in tokens] # 2 = <unk>
  padded_text = pad_sequences([indices], maxlen=MAX_LEN, padding='post')

  prediction = model.predict(padded_text, verbose=0)
  prob = prediction[0][0]

  if prob > 0.5:
    return f"긍정 (Positive) (확률: {prob*100:.2f}%)"
  else:
    return f"부정 (Negative) (확률: {(1-prob)*100:.2f}%)"

In [10]:

# 테스트
print("\n--- 새로운 리뷰 예측 테스트 ---")
test_review_1 = "This movie was absolutely fantastic, the best I have seen in years!"
test_review_2 = "It was a complete waste of time. The acting was terrible."
test_review_3 = "I didn't know what to expect, but this was surprisingly good."

print(f"Review 1: {test_review_1}")
print(f"Prediction: {predict_sentiment_keras(model, test_review_1)}")
print("-" * 20)
print(f"Review 2: {test_review_2}")
print(f"Prediction: {predict_sentiment_keras(model, test_review_2)}")
print("-" * 20)
print(f"Review 3: {test_review_3}")
print(f"Prediction: {predict_sentiment_keras(model, test_review_3)}")


--- 새로운 리뷰 예측 테스트 ---
Review 1: This movie was absolutely fantastic, the best I have seen in years!
Prediction: 긍정 (Positive) (확률: 76.77%)
--------------------
Review 2: It was a complete waste of time. The acting was terrible.
Prediction: 부정 (Negative) (확률: 67.54%)
--------------------
Review 3: I didn't know what to expect, but this was surprisingly good.
Prediction: 긍정 (Positive) (확률: 78.70%)
